In [108]:
import pandas as pd
from pprint import *
import numpy as np
# Display options
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land-monthly-means',
    {
        'format': 'grib',
        'year': '2018',
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'evaporation_from_bare_soil', 'evaporation_from_open_water_surfaces_excluding_oceans',
            'evaporation_from_the_top_of_canopy', 'evaporation_from_vegetation_transpiration', 'forecast_albedo',
            'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature',
            'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor',
            'lake_total_layer_temperature', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation',
            'potential_evaporation', 'runoff', 'skin_reservoir_content',
            'skin_temperature', 'snow_albedo', 'snow_cover',
            'snow_density', 'snow_depth', 'snow_depth_water_equivalent',
            'snow_evaporation', 'snowfall', 'snowmelt',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'sub_surface_runoff', 'surface_latent_heat_flux',
            'surface_net_solar_radiation', 'surface_net_thermal_radiation', 'surface_pressure',
            'surface_runoff', 'surface_sensible_heat_flux', 'surface_solar_radiation_downwards',
            'surface_thermal_radiation_downwards', 'temperature_of_snow_layer', 'total_evaporation',
            'total_precipitation', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
            'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
        ],
        'product_type': 'monthly_averaged_reanalysis',
    },
    'download.grib')


2021-03-03 16:11:51,329 INFO Welcome to the CDS
2021-03-03 16:11:51,333 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land-monthly-means
2021-03-03 16:11:51,459 INFO Request is queued
2021-03-03 16:16:09,422 WARNING Recovering from connection error [('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))], attemps 0 of 500
2021-03-03 16:16:09,423 WARNING Retrying in 120 seconds
2021-03-03 16:18:09,424 INFO Retrying now...
2021-03-03 16:20:09,913 INFO Request is running
2021-03-03 16:22:10,107 INFO Request is completed
2021-03-03 16:22:10,108 INFO Downloading https://download-0005.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1614784751.051031-6471-11-e1d7d418-e898-4129-be94-75a7add584b9.grib to download.grib (3.4G)
2021-03-03 16:28:16,651 INFO Download rate 9.6M/s                                                                         

Result(content_length=3672901440,content_type=application/x-grib,location=https://download-0005.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1614784751.051031-6471-11-e1d7d418-e898-4129-be94-75a7add584b9.grib)

In [257]:
# Europe hydroelectric generation dataset from pypsa-eur
df_hydro = pd.read_csv(r"C:\Users\Willem\pypsa-eur\data\bundle\EIA_hydro_generation_2000_2014.csv", 
                       sep = "\n")

original_head = list(df_hydro)[0]

df_hydro = df_hydro.iloc[:,0].str.split(',', expand=True)
display(df_hydro.head(10))
df_control = df_hydro.copy()

for col in list(df_hydro):
    df_hydro[col] = df_hydro[col].str.strip('"')

# Create countries list
l_countries = df_hydro.iloc[:,1].drop_duplicates(keep = "first").to_list()[:]
pprint(l_countries)

#display(df_hydro)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,http://www.eia.gov/beta/international/data/bro...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Wed Mar 01 2017 17:25:47 GMT+0100 (CET),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Source: U.S. Energy Information Administration,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,,,,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
4,,"""Electricity""","""Billion Kwh""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,,"""Generation""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,,"""Renewables""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,,"""Hydroelectricity""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,,"""Albania""","""Billion Kwh""",4.5,3.5,3.5,5.1,5.4,5.319,4.951,2.76,3.759,5.201,7.673,4.036,4.725,6.959,4.726
9,,"""Austria""","""Billion Kwh""",41,40,40,33,36,36.31,35.48,36.732,37.969,40.487,37.979,33.901,43.376,41.595,40.592


[None,
 '',
 'Electricity',
 'Generation',
 'Renewables',
 'Hydroelectricity',
 'Albania',
 'Austria',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Germany',
 'Gibraltar',
 'Greece',
 'Hungary',
 'Iceland',
 'Ireland',
 'Italy',
 'Kosovo',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Macedonia',
 'Malta',
 'Montenegro',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'United Kingdom']


In [158]:
df_input = pd.read_csv(r"C:\Users\Willem\Desktop\RWTH\Master\Semester 5\Masterarbeit\Backup code\additional data\INT-Export-03-09-2021_11-22-54.csv", 
                       sep = "\n")
df_input = df_input.iloc[:,0].str.split(",", expand = True)
df_input.columns = df_input.iloc[0]

l_dropcols = list(df_input)[2:-8] + list(df_input)[-3:]
df_input.drop(l_dropcols, axis =1, inplace = True)

# only look at hydroelectricity net generation
df_input = df_input[930:1160]
cols = list(df_input) 
cols[1] = "country"
cols = [col.replace('"', '') for col in cols]
df_input.columns = cols

for col in list(df_input):
    print(df_input[col][930])
    df_input[col] = df_input[col].str.strip('"')
    df_input[col] = df_input[col].str.strip(' ')

# only keep european countries
df_input["country"].loc[1083] = "Macedonia"
df_input = df_input[df_input["country"].isin(l_countries)]
df_input = df_input.iloc[:,1:]
#display(df_input)

# prepare dataframe for concatting
df_empty = pd.DataFrame(np.nan, index=[0, 1, 2, 3, 4, 5, 6, 7], columns=list(df_input))
df = pd.concat([df_empty, df_input], axis = 0, ignore_index = True)
df.loc[3] = list(df)
df.columns = ["xx",18,19,20,21,22]
df = df.drop([22,23])
ix = df.index.tolist()
# correct position of macedonia
ix = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
      16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30,
      31, 32, 33, 37, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
print(ix)
df = df.reindex(ix)
df.reset_index(inplace=True, drop = True)
display(df)


INTL.33-12-WORL-BKWH.A
"    World"
"3842.5372739"
"3989.76209413"
"4018.99163968"
"4155.57881582"
""
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


,xx,18,19,20,21,22
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,country,2015,2016,2017,2018,2019
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,Albania,5.83605,7.70418,4.47975,8.46648,5.15394
9,Austria,35.255,37.954,36.462,35.73,40.43655


In [276]:
# add new country specific time series to pypsa-eur input data 
df_all = pd.concat([df_control, df.iloc[:,1:]], axis = 1)
df_all = df_all.replace({np.nan: None})

for row in [14,22,33]:
    df_all.loc[row].replace("0" , '', inplace = True)  
    
# REVERSE read_csv formatting to get data into same weird format as df_hydro
# --------------


# add weird symbols
for col in list(df_all)[0:1]:
    df_all.iloc[:3,col] = '"' + df_all.iloc[:3,col].astype(str) + '"'

df_all.iloc[4:,0] = ""   


display(df_all)

# merge dataframe for export
df_all[original_head] = df_all[df_all.columns].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df_export = df_all[[original_head]]
df_all = df_all.drop(original_head, axis = 1)

display(df_export)
df_all.to_csv(path_or_buf = r"C:\Users\Willem\Desktop\RWTH\Master\Semester 5\Masterarbeit\Backup code\additional data\EIA_hydro_generation_2000_2019.csv",
             sep = ",",
             index = False,
             header = False,
             encoding = "utf-8",
             doublequote = True,
             #quotechar = '"',
             quotechar = "'")
###########
# Have to add "International_data" manually!!!
###########


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,"""http://www.eia.gov/beta/international/data/br...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"""Wed Mar 01 2017 17:25:47 GMT+0100 (CET)""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"""Source: U.S. Energy Information Administration""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,,,,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
4,,"""Electricity""","""Billion Kwh""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,,"""Generation""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,,"""Renewables""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,,"""Hydroelectricity""","""""",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,,"""Albania""","""Billion Kwh""",4.5,3.5,3.5,5.1,5.4,5.319,4.951,2.76,3.759,5.201,7.673,4.036,4.725,6.959,4.726,5.83605,7.70418,4.47975,8.46648,5.15394
9,,"""Austria""","""Billion Kwh""",41,40,40,33,36,36.31,35.48,36.732,37.969,40.487,37.979,33.901,43.376,41.595,40.592,35.255,37.954,36.462,35.73,40.43655


,International_data
0,"""http://www.eia.gov/beta/international/data/br..."
1,"""Wed Mar 01 2017 17:25:47 GMT+0100 (CET)"""
2,"""Source: U.S. Energy Information Administration"""
3,",,,2000,2001,2002,2003,2004,2005,2006,2007,200..."
4,",""Electricity"",""Billion Kwh"""
5,",""Generation"","""""
6,",""Renewables"","""""
7,",""Hydroelectricity"","""""
8,",""Albania"",""Billion Kwh"",4.5,3.5,3.5,5.1,5.4,5..."
9,",""Austria"",""Billion Kwh"",41,40,40,33,36,36.31,..."


In [278]:
df_hydro = pd.read_csv(r"C:\Users\Willem\pypsa-eur\data\bundle\EIA_hydro_generation_2000_2019.csv", 
                       sep = "\n")

original_head = list(df_hydro)[0]

df_hydro = df_hydro.iloc[:,0].str.split(',', expand=True)
display(df_hydro.head(10))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,http://www.eia.gov/beta/international/data/bro...,,,,,,,,,,,,,,,,,,,,,,
1,Wed Mar 01 2017 17:25:47 GMT+0100 (CET),,,,,,,,,,,,,,,,,,,,,,
2,Source: U.S. Energy Information Administration,,,,,,,,,,,,,,,,,,,,,,
3,,,,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
4,,"""Electricity""","""Billion Kwh""",,,,,,,,,,,,,,,,,,,,
5,,"""Generation""","""""",,,,,,,,,,,,,,,,,,,,
6,,"""Renewables""","""""",,,,,,,,,,,,,,,,,,,,
7,,"""Hydroelectricity""","""""",,,,,,,,,,,,,,,,,,,,
8,,"""Albania""","""Billion Kwh""",4.5,3.5,3.5,5.1,5.4,5.319,4.951,2.76,3.759,5.201,7.673,4.036,4.725,6.959,4.726,5.83605,7.70418,4.47975,8.46648,5.15394
9,,"""Austria""","""Billion Kwh""",41,40,40,33,36,36.31,35.48,36.732,37.969,40.487,37.979,33.901,43.376,41.595,40.592,35.255,37.954,36.462,35.73,40.43655
